In [91]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
import requests
import statsmodels.formula.api as sm

In [3]:
all2013=pd.read_csv('joined_shots_2013.csv')

In [299]:
!pip install rpy2

    100% |████████████████████████████████| 180kB 514kB/s 
  Running setup.py bdist_wheel for rpy2
  Stored in directory: /Users/harrisonchase/Library/Caches/pip/wheels/df/e3/df/8e2a19390b32c1def1764e580925de8e8a089a50d3c360e6ad
Successfully built rpy2


In [4]:
n_samples = all2013.shape[0]
all2013.head()

,Unnamed: 0,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,DRIBBLES,FGM,FINAL_MARGIN,GAME_CLOCK,GAME_ID,GAME_ID.1,LOCATION,MATCHUP,PERIOD,PERIOD.1,PTS,PTS_TYPE,SHOT_CLOCK,SHOT_DIST,SHOT_NUMBER,SHOT_RESULT,TOUCH_TIME,W,GRID_TYPE,GAME_EVENT_ID,PLAYER_ID,PLAYER_ID.1,PLAYER_NAME,TEAM_ID,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG
0,0,"Turner, Evan",202323,1.9,17,0,10,8:09,21300165,21300165,A,"NOV 20, 2013 - TOR @ PHI",2,2,0,2,9.8,7.7,1,missed,13.5,W,Shot Chart Detail,163,202779,202779,Dwight Buycks,1610612761,8,6,Missed Shot,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,-40,53,1,0
1,1,"Turner, Evan",202323,3.2,0,1,10,9:10,21300165,21300165,A,"NOV 20, 2013 - TOR @ PHI",4,4,3,3,8.1,23.5,2,made,1.4,W,Shot Chart Detail,449,202779,202779,Dwight Buycks,1610612761,9,7,Made Shot,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,26,182,195,1,1
2,2,"Carter-Williams, Michael",203487,3.5,0,0,10,8:36,21300165,21300165,A,"NOV 20, 2013 - TOR @ PHI",4,4,0,3,11.0,25.7,3,missed,1.3,W,Shot Chart Detail,452,202779,202779,Dwight Buycks,1610612761,8,33,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,26,159,206,1,0
3,3,"Brown, Lorenzo",203485,3.6,19,0,10,0:23,21300165,21300165,A,"NOV 20, 2013 - TOR @ PHI",4,4,0,3,9.0,24.7,4,missed,14.5,W,Shot Chart Detail,575,202779,202779,Dwight Buycks,1610612761,0,21,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,27,-15,274,1,0
4,0,"Cole, Norris",202708,3.7,0,1,-7,11:43,21300230,21300230,H,"NOV 29, 2013 - TOR vs. MIA",2,2,3,3,NaN,25.2,1,made,1.0,L,Shot Chart Detail,134,202779,202779,Dwight Buycks,1610612761,11,41,Made Shot,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,24,-149,198,1,1


In [298]:
plt.hist(np.log(1+all2013['DRIBBLES']))
plt.show()

In [499]:
y=all2013['SHOT_MADE_FLAG']
all2013['SHOT_CLOCK_ADJ']=60*all2013.MINUTES_REMAINING+all2013.SECONDS_REMAINING
all2013['SHOT_CLOCK_REAL']=[b if np.isnan(a) else a for a,b in zip(all2013['SHOT_CLOCK'],all2013['SHOT_CLOCK_ADJ'])]
X=all2013[['CLOSE_DEF_DIST','DRIBBLES','TOUCH_TIME','SHOT_DISTANCE']]

In [375]:
#zone = pd.get_dummies(all2013['SHOT_ZONE_AREA']).ix[:,1:]
#period = pd.get_dummies(all2013['PERIOD'])#.ix[:,1:]
#shottype = pd.get_dummies(all2013['SHOT_TYPE'])#.ix[:,1:]
#action = pd.get_dummies(all2013['ACTION_TYPE'])#.ix[:,1:]
#zonebasic = pd.get_dummies(all2013['SHOT_ZONE_BASIC'])#.ix[:,1:]
shotrange = pd.get_dummies(all2013['SHOT_ZONE_RANGE']).ix[:,1:]
#location = pd.get_dummies(all2013['LOCATION'])#.ix[:,1:]
#X=X.join(zone.join(period.join(shottype.join(zonebasic.join(shotrange.join(location.join(action)))))))
X=X.join(shotrange)

In [389]:
y=y[X.SHOT_CLOCK_REAL<25]
X=X[X.SHOT_CLOCK_REAL<25]
len(X.index)

194176

In [450]:
y_close=y[X.SHOT_DISTANCE>5]
X_close=X[X.SHOT_DISTANCE>5]

In [498]:
X.head()

,CLOSE_DEF_DIST,DRIBBLES,TOUCH_TIME,SHOT_DISTANCE,CLOSE,CLOSExCLOSE_DEF_DIST,CLOSExDRIBBLES,CLOSExTOUCH_TIME,CLOSExSHOT_DISTANCE,CLOSExCLOSE
0,1.9,17,13.5,6,0,0,0,0,0,0
1,3.2,0,1.4,26,0,0,0,0,0,0
2,3.5,0,1.3,26,0,0,0,0,0,0
3,3.6,19,14.5,27,0,0,0,0,0,0
4,3.7,0,1.0,24,0,0,0,0,0,0


In [501]:
X_new=X.copy()
X_new['CLOSE']=[1 if x<4 else 0 for x in X.SHOT_DISTANCE]
cols = X.columns
print cols
for col in cols:
    X_new['CLOSEx'+col] = X_new['CLOSE']*X[col]
X_new.head()

Index([u'CLOSE_DEF_DIST', u'DRIBBLES', u'TOUCH_TIME', u'SHOT_DISTANCE'], dtype='object')


,CLOSE_DEF_DIST,DRIBBLES,TOUCH_TIME,SHOT_DISTANCE,CLOSE,CLOSExCLOSE_DEF_DIST,CLOSExDRIBBLES,CLOSExTOUCH_TIME,CLOSExSHOT_DISTANCE
0,1.9,17,13.5,6,0,0,0,0,0
1,3.2,0,1.4,26,0,0,0,0,0
2,3.5,0,1.3,26,0,0,0,0,0
3,3.6,19,14.5,27,0,0,0,0,0
4,3.7,0,1.0,24,0,0,0,0,0


In [254]:
y=all2013['SHOT_MADE_FLAG']
X=all2013[['CLOSE_DEF_DIST','DRIBBLES','SHOT_DIST','TOUCH_TIME','SHOT_DISTANCE']]
"""X['LOCATION']=X['LOCATION'].astype('category')
X['PERIOD']=X['PERIOD'].astype('category')
X['SHOT_TYPE']=X['SHOT_TYPE'].astype('category')
X['SHOT_ZONE_BASIC']=X['SHOT_ZONE_BASIC'].astype('category')
X['SHOT_ZONE_AREA']=X['SHOT_ZONE_AREA'].astype('category')
X['SHOT_ZONE_RANGE']=X['SHOT_ZONE_RANGE'].astype('category')"""


"X['LOCATION']=X['LOCATION'].astype('category')\nX['PERIOD']=X['PERIOD'].astype('category')\nX['SHOT_TYPE']=X['SHOT_TYPE'].astype('category')\nX['SHOT_ZONE_BASIC']=X['SHOT_ZONE_BASIC'].astype('category')\nX['SHOT_ZONE_AREA']=X['SHOT_ZONE_AREA'].astype('category')\nX['SHOT_ZONE_RANGE']=X['SHOT_ZONE_RANGE'].astype('category')"

In [6]:
n_samples = X.shape[0]
X_train, y_train = X[:n_samples / 2], y[:n_samples / 2]
X_test, y_test = X[n_samples / 2:], y[n_samples / 2:]

In [138]:
from sklearn.linear_model import Lasso
from sklearn.cross_validation import KFold
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression

kf = KFold(X_train.shape[0], n_folds = 10,)
alphas = np.logspace(0, 3, num = 50, base = .01)
alphas

array([  1.00000000e+00,   7.54312006e-01,   5.68986603e-01,
         4.29193426e-01,   3.23745754e-01,   2.44205309e-01,
         1.84206997e-01,   1.38949549e-01,   1.04811313e-01,
         7.90604321e-02,   5.96362332e-02,   4.49843267e-02,
         3.39322177e-02,   2.55954792e-02,   1.93069773e-02,
         1.45634848e-02,   1.09854114e-02,   8.28642773e-03,
         6.25055193e-03,   4.71486636e-03,   3.55648031e-03,
         2.68269580e-03,   2.02358965e-03,   1.52641797e-03,
         1.15139540e-03,   8.68511374e-04,   6.55128557e-04,
         4.94171336e-04,   3.72759372e-04,   2.81176870e-04,
         2.12095089e-04,   1.59985872e-04,   1.20679264e-04,
         9.10298178e-05,   6.86648845e-05,   5.17947468e-05,
         3.90693994e-05,   2.94705170e-05,   2.22299648e-05,
         1.67683294e-05,   1.26485522e-05,   9.54095476e-06,
         7.19685673e-06,   5.42867544e-06,   4.09491506e-06,
         3.08884360e-06,   2.32995181e-06,   1.75751062e-06,
         1.32571137e-06,

In [491]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
hi=Lasso(alpha=.001)
#hi=LogisticRegression(penalty='l1',C=.001)
result = hi.fit(X_new,y)
y_hat=hi.predict(X)
print y_hat
print hi.score(X_new,y)
print accuracy_score(y_hat,y)
zip(hi.coef_[0],X_new.columns)


[ 0.37980652  0.32039614  0.32495456 ...,  0.36859133  0.6578924
  0.34763249]
0.0746038525262


ValueError: Can't handle mix of continuous and binary

In [508]:
dists=range(1,30,1)
alphas=[10,3,2,1,.1,.01,.001]
def get_dist(alpha):
    e_alphas = list()
    e_alphas_r = list()  #holds average r2 error
    for dist in dists:
        X_new=X.copy()
        X_new['CLOSE']=[1 if x<dist else 0 for x in X.SHOT_DISTANCE]
        cols = X.columns
        for col in cols:
            X_new['CLOSEx'+col] = X_new['CLOSE']*X[col]
        lasso = LogisticRegression(penalty='l2',C=alpha)
        err = list()
        err_2 = list()
        kf = KFold(len(y), n_folds=10)
        for tr_idx, tt_idx in kf:
            X_tr , X_tt = X_new.iloc[tr_idx], X_new.iloc[tt_idx]
            y_tr, y_tt = y.iloc[tr_idx], y.iloc[tt_idx]
            lasso.fit(X_tr, y_tr)
            y_hat = [x[1] for x in lasso.predict_proba(X_tt)]
            err.append(lasso.score(X_tt,y_tt))
            err_2.append(np.average((y_hat - y_tt)**2))
        e_alphas.append(np.average(err))
        e_alphas_r.append(np.average(err_2))
    return e_alphas

In [452]:
def get_alpha(dist):
    y_close, X_close = y[X.SHOT_DISTANCE<=dist],X[X.SHOT_DISTANCE<=dist]
    y_far, X_far = y[X.SHOT_DISTANCE>dist],X[X.SHOT_DISTANCE>dist]
    close_pct = len(y_close)/float(len(y))
    e_alphas = list()
    e_alphas_r = list()
    for alpha in alphas:
        lasso = LogisticRegression(penalty='l2',C=alpha)
        err_close = list()
        err_2_close = list()
        err_far = list()
        err_2_far = list()
        kf_close = KFold(len(y_close), n_folds=10)
        kf_far = KFold(len(y_far), n_folds=10)
        for tr_idx, tt_idx in kf_close:
            X_tr , X_tt = X_close.iloc[tr_idx], X_close.iloc[tt_idx]
            y_tr, y_tt = y_close.iloc[tr_idx], y_close.iloc[tt_idx]
            lasso.fit(X_tr, y_tr)
            y_hat = [x[1] for x in lasso.predict_proba(X_tt)]
            err_close.append(lasso.score(X_tt,y_tt))
            err_2_close.append(np.average((y_hat - y_tt)**2))
        for tr_idx, tt_idx in kf_far:
            X_tr , X_tt = X_far.iloc[tr_idx], X_far.iloc[tt_idx]
            y_tr, y_tt = y_far.iloc[tr_idx], y_far.iloc[tt_idx]
            lasso.fit(X_tr, y_tr)
            y_hat = [x[1] for x in lasso.predict_proba(X_tt)]
            err_far.append(lasso.score(X_tt,y_tt))
            err_2_far.append(np.average((y_hat - y_tt)**2))
        e_alphas.append(np.average(err_close)*close_pct+np.average(err_far)*(1-close_pct))
        e_alphas_r.append(np.average(err_2_close)*close_pct+np.average(err_2_far)*(1-close_pct))
    return e_alphas

In [509]:
accuracy={}
for alpha in alphas:
    print alpha
    diff_dist=get_dist(alpha)
    for index,dist in enumerate(dists):
        accuracy[(alpha,dist)]=diff_dist[index]

10
3
2
1
0.1
0.01
0.001


In [513]:
import operator
sorted_x = sorted(accuracy.items(), key=operator.itemgetter(1))
sorted_x

[((0.1, 28), 0.60538011241616707),
 ((3, 28), 0.60538011266377179),
 ((10, 28), 0.60538508803560709),
 ((0.01, 29), 0.60539006588349054),
 ((10, 29), 0.60539504125532573),
 ((3, 29), 0.60540001687476563),
 ((1, 28), 0.60540001687476575),
 ((1, 29), 0.6054000173699754),
 ((2, 29), 0.60540499274181048),
 ((2, 28), 0.60540996836125049),
 ((0.01, 28), 0.60541494645673899),
 ((0.1, 29), 0.60542987405787341),
 ((0.1, 27), 0.60543982603956803),
 ((10, 27), 0.6054647061176065),
 ((2, 27), 0.60546968223225628),
 ((3, 27), 0.60546968223225628),
 ((1, 27), 0.60546968272746604),
 ((0.01, 27), 0.60546968297507087),
 ((0.01, 26), 0.60587274157636417),
 ((0.1, 26), 0.60588766892989387),
 ((1, 26), 0.60597226040288987),
 ((2, 26), 0.60597723676514459),
 ((10, 26), 0.60600211659557846),
 ((3, 26), 0.60601704493952746),
 ((0.001, 27), 0.6060817294778762),
 ((0.001, 29), 0.60611656574689188),
 ((0.001, 28), 0.60611656624210153),
 ((0.001, 26), 0.60633053545796511),
 ((0.01, 25), 0.60702220152839936),
 ((

In [370]:
clf=SGDClassifier(penalty='elasticnet',loss='log',alpha=3)
clf.fit(X,y)
print clf.score(X,y)
print clf.predict_proba(X)

0.59383754944
[[ 0.50107504  0.49892496]
 [ 0.56061313  0.43938687]
 [ 0.57236403  0.42763597]
 ..., 
 [ 0.60845297  0.39154703]
 [ 0.48734425  0.51265575]
 [ 0.5936727   0.4063273 ]]


In [9]:
result.coef_

array([[ 0.09699993,  0.04662496, -0.01978541, -0.09407258, -0.03858662]])

In [262]:
list(X.columns)

['CLOSE_DEF_DIST',
 'DRIBBLES',
 'SHOT_DIST',
 'TOUCH_TIME',
 'SHOT_DISTANCE',
 'Back Court(BC)',
 'Center(C)',
 'Left Side Center(LC)',
 'Left Side(L)',
 'Right Side Center(RC)',
 'Right Side(R)',
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 '2PT Field Goal',
 '3PT Field Goal',
 'Above the Break 3',
 'Backcourt',
 'In The Paint (Non-RA)',
 'Left Corner 3',
 'Mid-Range',
 'Restricted Area',
 'Right Corner 3',
 '16-24 ft.',
 '24+ ft.',
 '8-16 ft.',
 'Back Court Shot',
 'Less Than 8 ft.',
 'A',
 'H',
 'Alley Oop Dunk Shot',
 'Alley Oop Layup shot',
 'Driving Bank Hook Shot',
 'Driving Bank shot',
 'Driving Dunk Shot',
 'Driving Finger Roll Layup Shot',
 'Driving Hook Shot',
 'Driving Jump shot',
 'Driving Layup Shot',
 'Driving Reverse Layup Shot',
 'Driving Slam Dunk Shot',
 'Dunk Shot',
 'Fadeaway Bank shot',
 'Fadeaway Jump Shot',
 'Finger Roll Layup Shot',
 'Floating Jump shot',
 'Hook Bank Shot',
 'Hook Shot',
 'Jump Bank Hook Shot',
 'Jump Bank Shot',
 'Jump Hook Shot',
 'Jump Shot',

In [379]:
X['SHOT_DISTxCLOSE_DEF_DIST'] = X['SHOT_DIST']*X['CLOSE_DEF_DIST']
X['CS']=X['DRIBBLES']==0
X['REALLY_CLOSE']=X['SHOT_DIST']<=4

In [245]:
alphas=[.1,.08,.06,.05]
e_alphas = list()
e_alphas_r = list()  #holds average r2 error
for alpha in alphas:
    lasso = SGDClassifier(penalty='elasticnet',loss='log',alpha=alpha)
    err = list()
    err_2 = list()
    for tr_idx, tt_idx in kf:
        X_tr , X_tt = X_train.loc[tr_idx], X_train.loc[tt_idx]
        y_tr, y_tt = y_train.loc[tr_idx], y_train.loc[tt_idx]
        lasso.fit(X_tr, y_tr)
        y_hat = [x[1] for x in lasso.predict_proba(X_tt)]
        err_2.append(lasso.score(X_tt,y_tt))
        err.append(np.average((y_hat - y_tt)**2))
    e_alphas.append(np.average(err))
    e_alphas_r.append(np.average(err_2))

In [246]:
e_alphas_r


[0.60792596373312169,
 0.60712971111141401,
 0.60631368178885237,
 0.60535825391093201]

In [255]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
new_X=pd.DataFrame(poly.fit_transform(X))

In [256]:
new_X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,1,1.9,17,7.7,13.5,6,3.61,32.3,14.63,25.65,11.4,289,130.9,229.5,102,59.29,103.95,46.2,182.25,81.0,36
1,1,3.2,0,23.5,1.4,26,10.24,0.0,75.20,4.48,83.2,0,0.0,0.0,0,552.25,32.90,611.0,1.96,36.4,676
2,1,3.5,0,25.7,1.3,26,12.25,0.0,89.95,4.55,91.0,0,0.0,0.0,0,660.49,33.41,668.2,1.69,33.8,676
3,1,3.6,19,24.7,14.5,27,12.96,68.4,88.92,52.20,97.2,361,469.3,275.5,513,610.09,358.15,666.9,210.25,391.5,729
4,1,3.7,0,25.2,1.0,24,13.69,0.0,93.24,3.70,88.8,0,0.0,0.0,0,635.04,25.20,604.8,1.00,24.0,576


In [410]:
lasso = SGDClassifier(penalty='elasticnet',loss='log',alpha=.1)
result=lasso.fit(X_close,y_close)
result.score(X_close,y_close)

0.63943891848612688

array([ 0.        ,  0.        ,  0.        , -0.17722361, -0.16257227,
        0.        ,  0.        ,  0.        ,  0.00897314,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       -0.004391  ,  0.        ,  0.00933107,  0.        ,  0.        ,
       -0.01396977])

In [273]:
zip(X.columns,result.coef_[0])

[('CLOSE_DEF_DIST', 0.059203984423060303),
 ('DRIBBLES', 0.0),
 ('SHOT_DIST', -0.024875865516824379),
 ('TOUCH_TIME', -0.04965403646615698),
 ('SHOT_DISTANCE', -0.027876054878132911),
 ('Back Court(BC)', 0.0),
 ('Center(C)', 0.0),
 ('Left Side Center(LC)', 0.0),
 ('Left Side(L)', 0.0),
 ('Right Side Center(RC)', 0.0),
 ('Right Side(R)', 0.0),
 (1.0, 0.0),
 (2.0, 0.0),
 (3.0, 0.0),
 (4.0, 0.0),
 (5.0, 0.0),
 (6.0, 0.0),
 (7.0, 0.0),
 ('2PT Field Goal', 0.0),
 ('3PT Field Goal', 0.0),
 ('Above the Break 3', 0.0),
 ('Backcourt', 0.0),
 ('In The Paint (Non-RA)', 0.0),
 ('Left Corner 3', 0.0),
 ('Mid-Range', 0.0),
 ('Restricted Area', 0.0),
 ('Right Corner 3', 0.0),
 ('16-24 ft.', 0.0),
 ('24+ ft.', 0.0),
 ('8-16 ft.', 0.0),
 ('Back Court Shot', 0.0),
 ('Less Than 8 ft.', 0.0),
 ('A', 0.0),
 ('H', 0.0),
 ('Alley Oop Dunk Shot', 0.0),
 ('Alley Oop Layup shot', 0.0),
 ('Driving Bank Hook Shot', 0.0),
 ('Driving Bank shot', 0.0),
 ('Driving Dunk Shot', 0.0),
 ('Driving Finger Roll Layup Shot',

In [ ]:
plt.figsize = (15,10)
fig = plt.figure()     
ax = fig.add_subplot(111)
ax.plot(alphas, e_alphas, 'b-')
ax.plot(alphas, e_alphas_r, 'g--')
ax.set_xlabel("alpha")
plt.show()

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.cross_validation import KFold
from matplotlib import pyplot as plt

kf = KFold(X_train.shape[0], n_folds = 10,)


In [89]:
def vizualize(name,y,cuts):
    x=X[name]
    rang = np.linspace(min(x),max(x),cuts)
    count = []
    pct = []
    for index,val in enumerate(rang):
        if index==cuts-2:
            a=x[(x>=val) & (x<=rang[index+1])]
            b=y[(x>=val) & (x<=rang[index+1])]
            count.append(len(a)/float(len(y)))
            pct.append(sum(b)/float(max(1,len(b)))) 
        elif index<cuts-1:
            a=x[(x>=val) & (x<rang[index+1])]
            b=y[(x>=val) & (x<rang[index+1])]
            count.append(len(a)/float(len(y)))
            pct.append(sum(b)/float(max(1,len(b))))    
    output= list(rang)[0:cuts-1],count,pct
    fig, ax1 = plt.subplots()
    ax1.plot(output[0],output[1],color='b')
    ax1.set_xlabel(name)
    ax1.set_ylabel('Frequency', color='b')
    for tl in ax1.get_yticklabels():
        tl.set_color('b')
    ax2 = ax1.twinx()
    ax2.plot(output[0],output[2],color='r')
    ax2.set_ylabel('Shooting Percentage', color='r')
    for tl in ax2.get_yticklabels():
        tl.set_color('r')
    plt.suptitle("Exploration of the variable " + name)
    plt.show()

In [391]:
output = vizualize('SHOT_CLOCK_REAL',y,50)

In [402]:
plt.scatter(X['SHOT_DIST'],X['SHOT_DISTANCE'],alpha=.2)
plt.show()

In [403]:
huh=all2013[(all2013.SHOT_DIST>20)&(all2013.SHOT_DISTANCE<3)]
huh.head()

,Unnamed: 0,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,DRIBBLES,FGM,FINAL_MARGIN,GAME_CLOCK,GAME_ID,GAME_ID.1,LOCATION,MATCHUP,PERIOD,PERIOD.1,PTS,PTS_TYPE,SHOT_CLOCK,SHOT_DIST,SHOT_NUMBER,SHOT_RESULT,TOUCH_TIME,W,GRID_TYPE,GAME_EVENT_ID,PLAYER_ID,PLAYER_ID.1,PLAYER_NAME,TEAM_ID,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,SHOT_CLOCK_ADJ,SHOT_CLOCK_REAL
82,5,"Gortat, Marcin",101162,2.5,0,1,-1,3:43,21300361,21300361,H,"DEC 16, 2013 - NYK vs. WAS",1,1,2,2,12.5,20.1,6,made,3.3,L,Shot Chart Detail,74,200745,200745,Andrea Bargnani,1610612752,3,39,Made Shot,Driving Reverse Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,-8,4,1,1,219,12.5
621,20,"Mahinmi, Ian",101133,2.3,0,0,-5,7:53,21300739,21300739,A,"FEB 07, 2014 - POR @ IND",4,4,0,2,20.2,20.8,21,missed,3.5,L,Shot Chart Detail,433,200746,200746,LaMarcus Aldridge,1610612757,7,56,Missed Shot,Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,2,12,1,0,476,20.2
4817,5,"Outlaw, Travis",2566,14.4,0,0,14,0:24,21300201,21300201,H,"NOV 24, 2013 - LAL vs. SAC",4,4,0,2,13.0,26.3,6,missed,1.7,W,Shot Chart Detail,484,200761,200761,Shawne Williams,1610612747,0,14,Missed Shot,Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,9,-1,1,0,14,13.0
5146,4,"Anderson, James",202341,3.7,1,0,10,8:27,21301034,21301034,H,"MAR 22, 2014 - CHI vs. PHI",3,3,0,2,19.7,22.8,5,missed,1.0,W,Shot Chart Detail,322,2399,2399,Mike Dunleavy,1610612741,8,31,Missed Shot,Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,-5,26,1,0,511,19.7
5700,5,"Teague, Jeff",201952,2.2,4,1,-8,2:30,21301166,21301166,A,"APR 09, 2014 - BOS @ ATL",2,2,2,2,19.0,25.3,6,made,4.3,L,Shot Chart Detail,218,200765,200765,Rajon Rondo,1610612738,2,34,Made Shot,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,12,12,1,1,154,19.0


In [309]:
from sklearn.feature_selection import RFE
estimator = LogisticRegression()
selector = RFE(estimator, 5, step=1)
selector = selector.fit(X, y)

In [153]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
X_new = SelectKBest(f_regression, k=10).fit_transform(X, y)
pd.DataFrame(X_new).head()

,0,1,2,3,4,5,6,7,8,9
0,7.7,6,1,0,0,0,1,0,0,1
1,23.5,26,0,1,0,1,0,0,0,1
2,25.7,26,0,1,0,1,0,0,0,1
3,24.7,27,1,1,0,1,0,0,0,1
4,25.2,24,0,1,0,1,0,0,0,1


In [386]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
clf = LassoCV()
# Set a minimum threshold of 0.25
sfm = SelectFromModel(clf, threshold=0.25)
sfm.fit(X, y)
n_features = sfm.transform(X).shape[1]

ImportError: cannot import name SelectFromModel

In [312]:
result = Lasso(alpha=.1).fit(X,y)

In [313]:
zip(X.columns,result.coef_)

[('CLOSE_DEF_DIST', 0.0),
 ('DRIBBLES', -0.0),
 ('SHOT_DIST', -0.0022926497236120369),
 ('TOUCH_TIME', -0.0),
 ('SHOT_DISTANCE', -0.009013933039435337),
 ('24+ ft.', 0.0),
 ('8-16 ft.', -0.0),
 ('Back Court Shot', -0.0),
 ('Less Than 8 ft.', 0.0),
 ('SHOT_DISTxCLOSE_DEF_DIST', 0.00039232346591208249),
 ('CS', 0.0),
 ('REALLY_CLOSE', 0.0)]

In [102]:
set(all2013['SHOT_ZONE_AREA'])

{'Back Court(BC)',
 'Center(C)',
 'Left Side Center(LC)',
 'Left Side(L)',
 'Right Side Center(RC)',
 'Right Side(R)'}

In [99]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
vec.fit_transform(all2013['SHOT_ZONE_AREA']).toarray()

AttributeError: 'str' object has no attribute 'iteritems'

In [64]:
plt.plot(output[0],output[1])
plt.show()